<a href="https://colab.research.google.com/github/hypo69/hypotez/blob/master/SANDBOX/davidka/LLM-HOWTO/LLM_%D1%88%D0%B0%D0%B3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # Для обработки пропусков
from scipy.sparse import hstack # Для соединения разреженных матриц (от TF-IDF) с плотными
import os
import zipfile
import json
from google.colab import drive


# 1. Подключение к Google Drive
try:
    drive.mount('/content/drive')
    print("Google Drive успешно подключен.")
except Exception as e:
    print(f"Ошибка подключения Google Drive: {e}")
    # Если не удалось подключить диск, дальнейшее выполнение бессмысленно
    raise SystemExit("Не удалось подключить Google Drive. Проверьте разрешения.")

# 2. Чтение файла json
#data:dict = json.loads('content/drive/hypo69/train.json')

file_path = '/content/drive/MyDrive/hypo69/train.csv'

try:
    # Пытаемся прочитать CSV-файл
    df = pd.read_csv(file_path)

    # Показываем первые несколько строк DataFrame для проверки
    print(df.head())

except FileNotFoundError:
    print(f"Файл не найден по пути: {file_path}")
    print("Убедитесь, что путь указан верно и файл существует в вашем Google Drive.")
except Exception as e:
    print(f"Произошла ошибка при чтении файла: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive успешно подключен.
                                                html  \
0  <div class="sticky"> <div class="w-full" id="f...   
1  <main class="w-full"><span> <div class="hidden...   
2  <main class="w-full"><span> <div class="hidden...   
3  <main class="w-full"><span><div class="grid"> ...   
4  <main class="w-full"><span><div class="grid"> ...   

                                                text  \
0     Follow Us\nServices\nCompany\nHave A Question?   
1  Multifunction Process Calibrators\nAdditel’s n...   
2  Accessories & Tools\nOur comprehensive collect...   
3  Additel 161\nIntelligent Digital Pressure Modu...   
4  Additel 878\nReference Dry Well Calibrator\nRe...   

                               title meta_og_title meta_name_title  \
0                           Products           NaN             NaN   
1  Multifunction Process 

In [ ]:


# --- Загрузка  DataFrame ---
# df = pd.read_csv('your_data.csv')
# Для примера создадим DataFrame:

df.loc[1, 'price'] = np.nan
df.loc[2, 'supplier'] = np.nan
df.loc[3, 'text'] = np.nan

# --- 1. Выбор признаков и цели ---
# Объединим несколько текстовых полей для лучшего представления
df['combined_text'] = df['title'].fillna('') + " " + \
                      df['text'].fillna('') + " " + \
                      df['description'].fillna('') + " " + \
                      df['meta_description'].fillna('')

text_features = 'combined_text'
categorical_features = ['supplier', 'page_type'] # Используем page_type как признак
numerical_features = ['price']
target_column = 'category' # Что будем предсказывать

# --- Подготовка X и y ---
X = df[numerical_features + categorical_features + [text_features]]
y_raw = df[target_column]

# --- 2. Предобработка целевой переменной (y) ---
# Кодируем текстовые метки категорий в числа
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_raw)
num_classes = len(label_encoder.classes_)
print(f"Классы для предсказания ({target_column}): {label_encoder.classes_}")
print(f"Количество уникальных классов: {num_classes}")

# Преобразуем в one-hot encoding для нейронной сети
# [[1,0,0], [0,1,0], ...]
y_one_hot = np.eye(num_classes)[y_encoded]

# --- 3. Предобработка признаков (X) ---
# Создаем конвейеры для каждого типа данных

# Для числовых: заполнение пропусков медианой, затем масштабирование
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Заполняем пропуски
    ('scaler', StandardScaler())
])

# Для категориальных: заполнение пропусков наиболее частым значением, затем one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Заполняем пропуски
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # handle_unknown='ignore' для новых категорий в тесте
])

# Для текстовых: TF-IDF (SimpleImputer не нужен, т.к. fillna('') сделали выше)
# Если бы не было fillna, то TfidfVectorizer упал бы на NaN.
# Можно было бы сделать:
# text_transformer = Pipeline(steps=[
# ('imputer', SimpleImputer(strategy='constant', fill_value='')), # Это если бы не делали df['combined_text'].fillna('')
# ('tfidf', TfidfVectorizer(max_features=500, stop_words='english')) # max_features ограничивает размер словаря
# ])
# Но так как мы уже обработали NaN при создании combined_text:
text_transformer = TfidfVectorizer(max_features=100, stop_words=None) # Уменьшил max_features для примера, None для русского

# Используем ColumnTransformer для применения разных трансформаций к разным колонкам
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('text', text_transformer, text_features) # text_features - это имя одной колонки 'combined_text'
    ],
    remainder='drop' # 'passthrough' если хотите оставить остальные колонки без изменений (но они должны быть числовыми)
)

# Применяем предобработку к X
X_processed = preprocessor.fit_transform(X)

# X_processed может быть разреженной матрицей (sparse matrix) из-за TF-IDF и OneHotEncoder.
# Ваша нейросеть ожидает плотный массив numpy.
if hasattr(X_processed, "toarray"): # Проверка, является ли матрица разреженной
    X_processed_dense = X_processed.toarray()
else:
    X_processed_dense = X_processed

print(f"Размер обработанных признаков X: {X_processed_dense.shape}")
print(f"Размер обработанной целевой переменной y: {y_one_hot.shape}")

# --- 4. Разделение на обучающую и тестовую выборки (опционально, но рекомендуется) ---
X_train, X_test, y_train, y_test = train_test_split(X_processed_dense, y_one_hot, test_size=0.2, random_state=42)

# --- 5. Подготовка данных для вашей нейронной сети ---
# Ваша сеть ожидает формат: [[входные_данные], ожидаемый_выход]
train_dataset_nn = []
for i in range(X_train.shape[0]):
    train_dataset_nn.append([X_train[i].tolist(), y_train[i].tolist()])

test_dataset_nn_inputs = [] # Только входы для тестирования в вашей функции test
for i in range(X_test.shape[0]):
    test_dataset_nn_inputs.append(X_test[i].tolist())


# --- 6. Создание и обучение вашей нейронной сети ---
input_layer_size = X_processed_dense.shape[1]  # Количество признаков после обработки
output_layer_size = num_classes                # Количество классов для предсказания

# Убедитесь, что ваш класс NeuroNetwork импортирован
# from your_neuro_network_module import NeuroNetwork

# Создаем экземпляр сети
# (предполагается, что код NeuroNetwork находится в том же файле или импортирован)
nn = NeuroNetwork(input_l_size=input_layer_size,
                  output_l_size=output_layer_size,
                  hidden_layers_count=1, # Можете настроить
                  learning_rate=0.01) # Возможно, потребуется подстроить

# Обучение сети
nn.train(train_dataset_nn, epochs=500) # epochs тоже подбирается

# --- 7. Тестирование сети ---
print("\n--- Тестирование на отложенной выборке ---")
# Ваша функция test ожидает список входов. Мы также знаем правильные ответы (y_test)
# для сравнения.
correct_predictions = 0
for i in range(len(test_dataset_nn_inputs)):
    input_sample = test_dataset_nn_inputs[i]
    true_label_one_hot = y_test[i]
    true_label_index = np.argmax(true_label_one_hot) # Индекс истинного класса

    nn.set_input_data(input_sample)
    prediction_probs = nn.get_prediction() # Вероятности для каждого класса
    predicted_label_index = np.argmax(prediction_probs) # Индекс предсказанного класса

    true_class_name = label_encoder.inverse_transform([true_label_index])[0]
    predicted_class_name = label_encoder.inverse_transform([predicted_label_index])[0]

    print(f"Образец {i+1}: Истинный класс: {true_class_name}, Предсказано: {predicted_class_name} (probs: {[f'{p:.2f}' for p in prediction_probs]})")
    if predicted_label_index == true_label_index:
        correct_predictions += 1

accuracy = correct_predictions / len(test_dataset_nn_inputs)
print(f"\nТочность на тестовой выборке: {accuracy:.4f}")


# --- ВАЖНО: Сохранение и загрузка предобработчиков ---
# Если вы сохраняете модель нейросети, вам также нужно сохранить
# `preprocessor` и `label_encoder`, чтобы применять их к новым данным перед предсказанием.
import joblib
joblib.dump(preprocessor, 'my_preprocessor.pkl')
joblib.dump(label_encoder, 'my_label_encoder.pkl')
nn.save_model('my_pandas_nn_model.pkl')

print("\nПредобработчик, кодировщик меток и модель сохранены.")

# --- Пример загрузки и использования ---
# loaded_preprocessor = joblib.load('my_preprocessor.pkl')
# loaded_label_encoder = joblib.load('my_label_encoder.pkl')
# loaded_nn_model = NeuroNetwork.load_model('my_pandas_nn_model.pkl')
#
# if loaded_nn_model and loaded_preprocessor and loaded_label_encoder:
#     print("\nМодель и обработчики загружены.")
    # Для предсказания на новых данных:
    # new_data_df = pd.DataFrame(...) # Ваши новые данные в том же формате, что и X
    # new_X_processed = loaded_preprocessor.transform(new_data_df)
    # if hasattr(new_X_processed, "toarray"): new_X_processed_dense = new_X_processed.toarray()
    # else: new_X_processed_dense = new_X_processed
    #
    # for sample in new_X_processed_dense:
    #     loaded_nn_model.set_input_data(sample.tolist())
    #     prediction = loaded_nn_model.get_prediction()
    #     predicted_label_idx = np.argmax(prediction)
    #     predicted_category = loaded_label_encoder.inverse_transform([predicted_label_idx])[0]
    #     print(f"Предсказание для нового образца: {predicted_category}")

Классы для предсказания (category): ['Bath Thermostats' 'Connectors' 'Furniture Assembly Components'
 'Limit Switch' 'Metal Nameplates' 'Plastic Components' 'Terminal Blocks'
 'Toiletry Bag' 'Turning Packages' 'Ultrafiltration Membrane' nan]
Количество уникальных классов: 11


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: '£30.44'